# Extending JWST PSFs

This tutorial demonstrates how to build extended JWST ePSF grids using `mophongo.jwst_psf`. It converts a directory of standard PSF grids into extended versions suitable for modelling wide halos.

In [ ]:
from pathlib import Path
import os
import logging
from photutils.psf import STDPSFGrid 
from mophongo import jwst_psf

logging.basicConfig(level=logging.WARNING)



The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol


## create several ST / Webb PSFs for every detector listed 

* search wcs_csv rate files produced by grizli to see which detectors
* do load_wss_opd_by_date closest to modal observation data of the mosaic 
* produce 4x oversampled (OS4) and detector sampled (DET) 
* produce single PSFs (center of detector) and square sampled grids (3x3 in 770, 5x5 in 444)



In [ ]:
from pathlib import Path
from mophongo.jwst_psf  import psf_grid_from_csv

data_dir = Path('../data')
psf_dir = data_dir / 'PSF'
filters =  [('f115w', 2.0, [1,25]), 
            ('f444w', 4.0, [1,25]), 
            ('f770w', 8.0, [1,9])]

pf = 'UDS'
for filt, fov, ngrid in filters:
    csv = data_dir / f'uds-test-{filt}_wcs.csv'
    for size in ngrid:
        print(csv,filt, fov, size)
        psf_grid_from_csv(csv, fov_arcsec=fov, num_psfs=size, prefix=pf, outdir=psf_dir,  
                          postfix=f'DET_GRID{size}', use_detsampled_psf=True, save=True)
        psf_grid_from_csv(csv, fov_arcsec=fov, num_psfs=size, prefix=pf, outdir=psf_dir, 
                          postfix=f'OS4_GRID{size}',save=True)
    

### Oversample DETECTOR sampled PSFs 

So that it remains compatible with DrizzlePSF which has x4 oversampling hard coded

In [ ]:
from pathlib import Path
from astropy.io import fits
from scipy.ndimage import zoom
import numpy as np

psf_dir = Path('../data/PSF')
oversample_factor = 4

for f in psf_dir.glob('*DET*GRID*.fits'):
    outfile = f.with_name(f.stem + f'_OS{oversample_factor}.fits')
    with fits.open(f) as hdul:
        data = hdul[0].data  # shape (Npsf, Y, X)
        header = hdul[0].header.copy()
    if data.ndim == 2:
        data = data[np.newaxis, ...]
    oversampled = zoom(data, (1, oversample_factor, oversample_factor), order=3)
    fits.writeto(outfile, oversampled, header, overwrite=True)
    print(f"Oversampled PSF saved to {outfile} with shape {oversampled.shape}")


../data/uds-test-f115w_wcs.csv f115w 2.0 1
OUTFILE: UDS_NRCA1_F115W_DET_GRID1.fits
Skipping UDS_NRCA1_F115W_DET_GRID1.fits (exists, overwrite=False)
OUTFILE: UDS_NRCA2_F115W_DET_GRID1.fits
Skipping UDS_NRCA2_F115W_DET_GRID1.fits (exists, overwrite=False)
OUTFILE: UDS_NRCA3_F115W_DET_GRID1.fits
Skipping UDS_NRCA3_F115W_DET_GRID1.fits (exists, overwrite=False)
OUTFILE: UDS_NRCA4_F115W_DET_GRID1.fits
Skipping UDS_NRCA4_F115W_DET_GRID1.fits (exists, overwrite=False)
OUTFILE: UDS_NRCB1_F115W_DET_GRID1.fits
Skipping UDS_NRCB1_F115W_DET_GRID1.fits (exists, overwrite=False)
OUTFILE: UDS_NRCB2_F115W_DET_GRID1.fits
Skipping UDS_NRCB2_F115W_DET_GRID1.fits (exists, overwrite=False)
OUTFILE: UDS_NRCB3_F115W_DET_GRID1.fits
Skipping UDS_NRCB3_F115W_DET_GRID1.fits (exists, overwrite=False)
OUTFILE: UDS_NRCB4_F115W_DET_GRID1.fits
Skipping UDS_NRCB4_F115W_DET_GRID1.fits (exists, overwrite=False)
OUTFILE: UDS_NRCA1_F115W_OS4_GRID1.fits
Skipping UDS_NRCA1_F115W_OS4_GRID1.fits (exists, overwrite=False)
OUT

  warnings.warn(



Wrote 25 PSFs ➜ ../data/PSF/UDS_NRCA1_F115W_DET_GRID25.fits
OUTFILE: UDS_NRCA2_F115W_DET_GRID25.fits

MAST OPD query around UTC: 59967.157610238
                        MJD: 59967.157610238

OPD immediately preceding the given datetime:
	URI:	 mast:JWST/product/R2023012202-NRCA3_FP1-1.fits
	Date (MJD):	 59965.8570
	Delta time:	 -1.3006 days

OPD immediately following the given datetime:
	URI:	 mast:JWST/product/R2023012303-NRCA3_FP1-1.fits
	Date (MJD):	 59967.6511
	Delta time:	 0.4935 days
User requested choosing OPD time closest in time to 59967.157610238, which is R2023012303-NRCA3_FP1-1.fits, delta time 0.493 days
Importing and format-converting OPD from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/MAST_JWST_WSS_OPDs/R2023012303-NRCA3_FP1-1.fits
Backing out SI WFE and OTE field dependence at the WF sensing field point (NRCA3_FP1)


  warnings.warn(



Wrote 25 PSFs ➜ ../data/PSF/UDS_NRCA2_F115W_DET_GRID25.fits
OUTFILE: UDS_NRCA3_F115W_DET_GRID25.fits

MAST OPD query around UTC: 59967.157610238
                        MJD: 59967.157610238

OPD immediately preceding the given datetime:
	URI:	 mast:JWST/product/R2023012202-NRCA3_FP1-1.fits
	Date (MJD):	 59965.8570
	Delta time:	 -1.3006 days

OPD immediately following the given datetime:
	URI:	 mast:JWST/product/R2023012303-NRCA3_FP1-1.fits
	Date (MJD):	 59967.6511
	Delta time:	 0.4935 days
User requested choosing OPD time closest in time to 59967.157610238, which is R2023012303-NRCA3_FP1-1.fits, delta time 0.493 days
Importing and format-converting OPD from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/MAST_JWST_WSS_OPDs/R2023012303-NRCA3_FP1-1.fits
Backing out SI WFE and OTE field dependence at the WF sensing field point (NRCA3_FP1)


...

Wrote 25 PSFs ➜ ../data/PSF/UDS_NRCB2_F115W_OS4_GRID25.fits
OUTFILE: UDS_NRCB3_F115W_OS4_GRID25.fits

MAST OPD query around UTC: 59967.157610238
                        MJD: 59967.157610238

OPD immediately preceding the given datetime:
	URI:	 mast:JWST/product/R2023012202-NRCA3_FP1-1.fits
	Date (MJD):	 59965.8570
	Delta time:	 -1.3006 days

OPD immediately following the given datetime:
	URI:	 mast:JWST/product/R2023012303-NRCA3_FP1-1.fits
	Date (MJD):	 59967.6511
	Delta time:	 0.4935 days
User requested choosing OPD time closest in time to 59967.157610238, which is R2023012303-NRCA3_FP1-1.fits, delta time 0.493 days
Importing and format-converting OPD from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/MAST_JWST_WSS_OPDs/R2023012303-NRCA3_FP1-1.fits
Backing out SI WFE and OTE field dependence at the WF sensing field point (NRCA3_FP1)


  warnings.warn(



Wrote 25 PSFs ➜ ../data/PSF/UDS_NRCB3_F115W_OS4_GRID25.fits
OUTFILE: UDS_NRCB4_F115W_OS4_GRID25.fits

MAST OPD query around UTC: 59967.157610238
                        MJD: 59967.157610238

OPD immediately preceding the given datetime:
	URI:	 mast:JWST/product/R2023012202-NRCA3_FP1-1.fits
	Date (MJD):	 59965.8570
	Delta time:	 -1.3006 days

OPD immediately following the given datetime:
	URI:	 mast:JWST/product/R2023012303-NRCA3_FP1-1.fits
	Date (MJD):	 59967.6511
	Delta time:	 0.4935 days
User requested choosing OPD time closest in time to 59967.157610238, which is R2023012303-NRCA3_FP1-1.fits, delta time 0.493 days
Importing and format-converting OPD from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/MAST_JWST_WSS_OPDs/R2023012303-NRCA3_FP1-1.fits
Backing out SI WFE and OTE field dependence at the WF sensing field point (NRCA3_FP1)


  warnings.warn(



Wrote 25 PSFs ➜ ../data/PSF/UDS_NRCB4_F115W_OS4_GRID25.fits
../data/uds-test-f444w_wcs.csv f444w 4.0 1
OUTFILE: UDS_NRCA5_F444W_DET_GRID1.fits

MAST OPD query around UTC: 59967.18805613
                        MJD: 59967.18805613

OPD immediately preceding the given datetime:
	URI:	 mast:JWST/product/R2023012202-NRCA3_FP1-1.fits
	Date (MJD):	 59965.8570
	Delta time:	 -1.3310 days

OPD immediately following the given datetime:
	URI:	 mast:JWST/product/R2023012303-NRCA3_FP1-1.fits
	Date (MJD):	 59967.6511
	Delta time:	 0.4630 days
User requested choosing OPD time closest in time to 59967.18805613, which is R2023012303-NRCA3_FP1-1.fits, delta time 0.463 days
Importing and format-converting OPD from /Users/ivo/Astro/PROJECTS/JWST/PSF/stpsf-data/MAST_JWST_WSS_OPDs/R2023012303-NRCA3_FP1-1.fits
Backing out SI WFE and OTE field dependence at the WF sensing field point (NRCA3_FP1)
Wrote 1 PSFs ➜ ../data/PSF/UDS_NRCA5_F444W_DET_GRID1.fits
OUTFILE: UDS_NRCB5_F444W_DET_GRID1.fits

MAST OPD query a

## Extending existing empirical ePSFs with wings from STPSF

Specify the directory containing the JWST STDPSF `*.fits` files. The extended PSFs will be written EXTENDED postfix


## Extending existing empirical ePSFs with wings from STPSF

Alternative approach. Use ePSF as core, extend with STPSF. The extended PSFs will be written EXTENDED postfix


In [ ]:

psf_dir = Path('../data/PSF')
outdir = psf_dir 
for f in psf_dir.rglob('*.fits'):
    outname = outdir / f.name.replace('.fits', '_EXTENDED.fits')  # extended psf
    outname_stpsf = outdir / f.name.replace('.fits', '_WEBB.fits') # webb stpsf

    if outname.exists() or 'EXTENDED' in f.name or 'WEBB' in f.name \
        or 'NRC' not in f.name or not '444' in f.name:
        continue

    epsf = STDPSFGrid(str(f))
    epsf_ext, st_psf = jwst_psf.make_extended_grid(epsf, Rmax=2.0, Rtaper=0.2, bg_pct=10, return_stpsf=True, verbose=True)

    os.makedirs(outdir, exist_ok=True)
    if not outname.exists(): 
        jwst_psf.write_stdpsf(outname, epsf_ext, overwrite=True, verbose=True)
    if not outname_stpsf.exists(): 
        jwst_psf.write_stdpsf(outname_stpsf, st_psf, overwrite=True, verbose=True)

Oversampled PSF saved to ../data/PSF/UDS_NRCA5_F444W_DET_GRID25_OS4.fits with shape (25, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCA3_F115W_DET_GRID1_OS4.fits with shape (1, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB4_F115W_DET_GRID1_OS4.fits with shape (1, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB1_F115W_DET_GRID1_OS4.fits with shape (1, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB5_F444W_DET_GRID25_OS4.fits with shape (25, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCA1_F115W_DET_GRID25_OS4.fits with shape (25, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB3_F115W_DET_GRID25_OS4.fits with shape (25, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCA3_F115W_DET_GRID25_OS4.fits with shape (25, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_NRCB5_F444W_DET_GRID1_OS4.fits with shape (1, 260, 260)
Oversampled PSF saved to ../data/PSF/UDS_MIRI_F770W_DET_GRID9_OS4.fits with shape (9, 292, 292)
Oversampled PSF saved